<a href="https://colab.research.google.com/github/KimHyungkeun/DE_Starterkit_Programmers/blob/week3_hw/week3_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Assignment 2
# ts별로 row_number를 매긴 후, 가장 빠른 순서가 먼저오는 테이블 a와 가장 늦은 순서가 먼저오는 테이블 b를 join 시킨다
# seq(row_number) 상으로는 가장 먼저오는 순서와 가장 늦게오는 순서가 모두 1이다.
# 따라서, WHERE 절에 seq를 1로 설정한 후에 user별 첫 채널과 늦은 채널을 고른다
%%sql

SELECT DISTINCT a.userid, a.channel as first_channel, b.channel as last_channel
FROM
(
SELECT userid, ts, channel, ROW_NUMBER() OVER (partition by userid order by ts) seq
FROM raw_data.user_session_channel usc
JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
) a 
JOIN
(
SELECT userid, ts, channel, ROW_NUMBER() OVER (partition by userid order by ts desc) seq
FROM raw_data.user_session_channel usc
JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
) b ON a.userid = b.userid and a.seq = b.seq
WHERE a.seq = 1
LIMIT 10;

In [ ]:
# Assignment 3
# 두 채널 a, b를 sessionid를 기준으로 join 시킨다.
# 이때, Gross Revenue가 가장 큰 user를 찾기위해서 refunded가 True인것을 모두 포함한 것들을 찾는다.
# NULL은 포함되지 않도록 한다
# amount를 내림차순으로 셋팅하여 TOP 10을 가려낸다
%%sql

SELECT userid, sum(CASE WHEN amount IS NULL THEN 0 ELSE amount END) as sum_amount
FROM raw_data.user_session_channel a
LEFT OUTER JOIN raw_data.session_transaction b
ON a.sessionid = b.sessionid AND refunded IS NOT NULL
GROUP BY userid
ORDER BY sum_amount DESC
LIMIT 10;

In [ ]:
# Assignment 4
# user_count_table은 해당 홈페이지에 한번이라도 방문이 기록된 유저 종류 수(uniqueUsers)와 방문이 기록된 유저들 중 session_transaction에 한번이라도 영향을 준 유저 종류 수(paidUsers)를 기록한 테이블이다
# revenue_count_table은 해당 홈페이지에서 refund를 포함한 모든 수입(grossRevenue)와 refund가 되지 않은 수입(netRevenue)를 기록한 테이블이다
# 두 테이블을 JOIN 시킨다
%%sql

CREATE TABLE hyungkeun_kim95.week3_assignment 
AS SELECT * FROM
(
  WITH user_count_table AS 
  (   SELECT A.year_month, A.channel, uniqueUsers, paidUsers
      FROM
      (
        (SELECT to_char(ts, 'YYYY-MM') as year_month, channel, COUNT(DISTINCT userid) as uniqueUsers
        FROM raw_data.session_timestamp a
        JOIN raw_data.user_session_channel b ON a.sessionid = b.sessionid 
        GROUP BY year_month, channel) A
        JOIN
        (SELECT to_char(ts, 'YYYY-MM') as year_month, channel, COUNT(DISTINCT userid) as paidUsers
        FROM raw_data.session_timestamp a
        JOIN raw_data.user_session_channel b ON a.sessionid = b.sessionid 
        LEFT OUTER JOIN raw_data.session_transaction c ON b.sessionid = c.sessionid
        WHERE c.sessionid IS NOT NULL
        GROUP BY year_month, channel) B
        ON A.year_month = B.year_month AND A.channel = B.channel 
      )
  ), revenue_count_table AS 
  (   SELECT C.year_month, C.channel, grossRevenue, netRevenue 
      FROM
      (
        (SELECT to_char(ts, 'YYYY-MM') as year_month, channel, sum(CASE WHEN amount IS NULL THEN 0 ELSE amount END) as grossRevenue
        FROM raw_data.session_timestamp a
        JOIN raw_data.user_session_channel b ON a.sessionid = b.sessionid 
        LEFT OUTER JOIN raw_data.session_transaction c ON b.sessionid = c.sessionid AND refunded IS NOT NULL
        GROUP BY year_month, channel) C
        JOIN
        (SELECT to_char(ts, 'YYYY-MM') as year_month, channel, sum(CASE WHEN amount IS NULL THEN 0 ELSE amount END) as netRevenue
        FROM raw_data.session_timestamp a
        JOIN raw_data.user_session_channel b ON a.sessionid = b.sessionid 
        LEFT OUTER JOIN raw_data.session_transaction c ON b.sessionid = c.sessionid AND refunded IS NOT NULL AND refunded IS FALSE
        GROUP BY year_month, channel) D
        ON C.year_month = D.year_month AND C.channel = D.channel
      )
  ) 
  SELECT E.year_month, E.channel, uniqueUsers, paidUsers, CAST(paidUsers AS DOUBLE PRECISION) / NULLIF(uniqueUsers, 0) as ConversionRate, grossRevenue, netRevenue
  FROM user_count_table E JOIN revenue_count_table F ON E.year_month = F.year_month AND E.channel = F.channel
) 